# Скачаем униграммы на букву d, нормализуем и пропустим через словарь из Викиданных

In [10]:
from google_ngram_downloader import readline_google_store
from string import punctuation
from pymystem3 import Mystem
import pandas as pd
import re
import csv

morph = Mystem()
punct = punctuation+'«»—…“”""*–'
russian = "[А-Яа-я]+"

In [3]:
def load_unigrams(my_len, my_indices):
    tags = ("_ADJ", "_ADP", "_ADV", "_CONJ", "_NOUN", "_NUM", "_PRT", "_VERB", "_X")
    fname, url, records = next(readline_google_store(ngram_len=1, lang='rus', indices=my_indices))
    record = next(records)
    with open('/Users/anyway/Desktop/ngrams/unigrams_' + my_indices + '.tsv', 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        count = 0
        while True:
            try:
                if record.year < 1918:
                    record = next(records)
                else:
                    if len(record.ngram.strip(punct)) > 2 and re.search(russian, record.ngram) and not record.ngram.endswith(tags):
                        writer.writerow([record.ngram,
                                        record.year, 
                                        record.match_count, 
                                        record.volume_count])
                        count += 1
                        record = next(records)
                    else:
                        record = next(records)
            except StopIteration:
                break
                print("StopIteration")
    return str(count) + " " + my_indices + " ngrams saved"

In [23]:
%%time
#загрузим униграммы на букву "Д"
load_unigrams(1, "d")

CPU times: user 2min 54s, sys: 4.46 s, total: 2min 59s
Wall time: 3min 21s


'10087582 d ngrams saved'

In [6]:
%%time
with open('/Users/anyway/Desktop/ngrams/unigrams_d.tsv', 'r') as f:
    unigrams = f.readlines()
    
for i, line in enumerate(unigrams):
    unigrams[i] = line.strip().split("\t")
    
unigrams[0] 

CPU times: user 45.1 s, sys: 1min 49s, total: 2min 34s
Wall time: 3min 3s


In [7]:
%%time
unique_unigrams = {line[0] for line in unigrams}
print(len(unique_unigrams))

231327
CPU times: user 2.68 s, sys: 4.5 s, total: 7.19 s
Wall time: 8.72 s


In [11]:
%%time
# создадим словарь, где ключ - униграмма, значение - лемма
punct_and_numbers = punct + '1234567890'
unique_unigrams = list(unique_unigrams)
lemmas_dict = {}
for unigram in unique_unigrams:
    clean_unigram = unigram.strip(punct_and_numbers)
    lemmas_dict[unigram] = ''.join(morph.lemmatize(clean_unigram)).strip().lower() 

CPU times: user 18.8 s, sys: 11.3 s, total: 30.1 s
Wall time: 1min 17s


In [18]:
lemmas_dict["древнеармянскими"]

'древнеармянский'

In [6]:
unigrams[:5]

[['Д.Боровского', '1980', '9', '1'],
 ['Д.Боровского', '1983', '1', '1'],
 ['Д.Боровского', '1988', '5', '3'],
 ['Д.Боровского', '1990', '13', '3'],
 ['Д.Боровского', '1993', '1', '1']]

In [12]:
# с помощью данного словаря заменим все униграммы на букву d на их леммы
lemmatized_unigrams = []
for line in unigrams:
    line[0] = lemmas_dict[line[0]]
    if len(line[0]) > 2: # отбрасываем униграммы в 1-2 буквы
        lemmatized_unigrams.append(line)

In [8]:
print(len(lemmatized_unigrams))
lemmatized_unigrams[:20]

10015616


[['д.боровский', '1980', '9', '1'],
 ['д.боровский', '1983', '1', '1'],
 ['д.боровский', '1988', '5', '3'],
 ['д.боровский', '1990', '13', '3'],
 ['д.боровский', '1993', '1', '1'],
 ['д.боровский', '1994', '3', '2'],
 ['д.боровский', '1995', '1', '1'],
 ['д.боровский', '1996', '3', '2'],
 ['д.боровский', '1997', '3', '3'],
 ['д.боровский', '1998', '1', '1'],
 ['д.боровский', '1999', '3', '2'],
 ['д.боровский', '2000', '6', '5'],
 ['д.боровский', '2001', '2', '2'],
 ['д.боровский', '2002', '2', '2'],
 ['д.боровский', '2003', '8', '4'],
 ['д.боровский', '2005', '3', '3'],
 ['д.боровский', '2006', '1', '1'],
 ['д.боровский', '2007', '1', '1'],
 ['д.боровский', '2008', '3', '3'],
 ['давль', '1930', '1', '1']]

In [19]:
# теперь нам нужно соединить частотность одинаковых униграмм по году 
lemmatized_unigrams = sorted(lemmatized_unigrams)

In [14]:
df = pd.DataFrame(lemmatized_unigrams, columns = ["unigram", "year", "match_count", "volume_count"])
df.head(100)

,unigram,year,match_count,volume_count
0,д'6йств1я,1930,2,1
1,д'6йств1я,1968,2,2
2,д'а,1918,1,1
3,д'а,1920,2,1
4,д'а,1922,1,1
5,д'а,1922,3,3
6,д'а,1923,1,1
7,д'а,1925,1,1
8,д'а,1925,2,2
9,д'а,1926,12,3


In [13]:
%%time

df["match_count"] = df["match_count"].astype(int)
df["volume_count"] = df["volume_count"].astype(int)

df_grouped = df.groupby(["unigram", "year"]).sum()

CPU times: user 4.25 s, sys: 3.56 s, total: 7.81 s
Wall time: 8.36 s


In [30]:
df_grouped.head(100)

match_count  volume_count
unigram   year                           
д'6йств1я 1930            2             1
          1968            2             2
д'а       1918            1             1
          1920            2             1
          1922            4             4
          1923            1             1
          1925            3             3
          1926           14             5
          1927            8             6
          1928            9             4
          1929           11             8
          1930            7             6
          1931           13            10
          1932           45             9
          1933            8             8
          1934            6             6
          1935           10             9
          1936            6             6
          1937            6             6
          1938            5             5
          1939           14            13
          1940            7             7
          1941            2             1
          1944            1             1
          1945            2             2
          1946            4             4
          1947           12            11
          1948           15            10
          1949           11             9
          1950           74            21
...                     ...           ...
          1991            8             8
          1992            9             9
          1993           89            12
          1994           12             7
          1995            6             6
          1996            8             7
          1997           30             5
          1998            9             7
          1999           25             7
          2000           12             9
          2001            7             6
          2002           19             4
          2003           18             7
          2004            8             7
          2005            3             2
          2006            4             4
          2007           31             6
          2008           12             3
          2009            1             1
д'абано   1923            2             2
          1936            3             1
          1947            7             7
          1958            7             3
          1960            3             3
          1961            1             1
          1962            1             1
          1963            2             2
          1966            2             1
          1967            3             3
          1969            1             1

[100 rows x 2 columns]

In [29]:
df["unigram"].nunique()

80554

In [15]:
unique = df["unigram"].unique()
unique = unique.tolist()

### Cоздадим словарь униграмм

In [2]:
%%time

# сначала прочтем нормализованные Викиданные из файла
def read_tsv(file):
    with open(file, "r", encoding="UTF-8") as f:
        f = f.readlines()
        print(len(f))
    data = []
    for line in f:
        line = line.strip('\n').split(' | ')
        data.append(line)
    return data

norm_data = read_tsv("/Users/anyway/Desktop/wikidata/norm-wikidata.tsv")

2234352
CPU times: user 6.93 s, sys: 2.12 s, total: 9.04 s
Wall time: 9.68 s


In [3]:
back_dict = {line[0]:set() for line in norm_data}

In [4]:
def update_dict(dict_, ngrams, Q):
    
    if isinstance(ngrams, str):
        ngram = ngrams
        if ngram not in dict_:
            dict_[ngram] = {Q}
        else:
            dict_[ngram].add(Q)
        back_dict[Q].add(ngram)

    if isinstance(ngrams, list):
        for ngram in ngrams:
            if ngram not in dict_:
                dict_[ngram] = {Q}
            else:
                dict_[ngram].add(Q)
            back_dict[Q].add(ngram)
            
    return 0

In [5]:
%%time
#создадим словарь униграмм
unigram_dict = {}
for line in norm_data:
    if len(line[1:]) == 1 and len(line[1].split()) == 1:
        Q, unigram = line[0], line[1]
        update_dict(unigram_dict, unigram, Q)
    elif len(line[1:]) > 1:
        for entity_name in line[1:]:
            entity_name_split = entity_name.split()
            if len(entity_name_split) == 1:
                Q, unigram = line[0], entity_name_split[0]
                update_dict(unigram_dict, unigram, Q)
len(unigram_dict)

CPU times: user 7.02 s, sys: 2.71 s, total: 9.73 s
Wall time: 10.2 s


In [16]:
# добавим в новый словарь те униграммы, для которых найдутся номера Q
unigrams_with_Q = {}
for word in unique:
    if word in unigram_dict:
        unigrams_with_Q[word] = tuple(unigram_dict[word])
        
len(unigrams_with_Q)

8734

In [17]:
unigrams_with_Q

{"д'акоста": ('Q555476',),
 "д'актиль": ('Q4172652',),
 "д'аламбер": ('Q153232',),
 "д'анж": ('Q21126312',),
 "д'анжу": ('Q21126313',),
 "д'аннунцио": ('Q193236',),
 "д'ансельм": ('Q1952844',),
 "д'арра": ('Q155013',),
 "д'арси": ('Q242280',),
 "д'артаньян": ('Q744183',),
 "д'ор": ('Q15720364',),
 "д'орсэ": ('Q23402',),
 "д'эстер": ('Q1461808',),
 "д'юрвиль": ('Q4172687',),
 "д'юрф": ('Q21449726', 'Q1272999', 'Q358491'),
 'д.и.яворницкий': ('Q1963094',),
 'д1м': ('Q4161422',),
 'даад': ('Q696935',),
 'даам': ('Q1100742',),
 'даан': ('Q1156469', 'Q570408'),
 'даар': ('Q157197',),
 'даб': ('Q949630', 'Q212688'),
 'даба': ('Q5207451', 'Q15008703', 'Q1588249', 'Q420917'),
 'дабагян': ('Q28947685',),
 'дабан': ('Q32713664',
  'Q60830678',
  'Q32713669',
  'Q32713674',
  'Q32713679',
  'Q16896557'),
 'дабат': ('Q32713827', 'Q3011586', 'Q23957653', 'Q32713814'),
 'дабач': ('Q1047025',),
 'дабашань': ('Q1156502',),
 'дабба': ('Q1165396', 'Q254042'),
 'даббах': ('Q1156519',),
 'даббо': ('Q12629